# Ensemble Analysis
#### This notebook reads in and formats outputs from ensemble simulations so that you can evaluate and plot


In [2]:

#Import packages
#import packages
import flopy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
import pandas as pd
import numpy as np
import os


#additional analysis tools
import flopy.utils.binaryfile as bf
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


np.set_printoptions(threshold=np.inf)

#jupyter specific--included to show plots in notebook
%matplotlib inline 

#Setup static variables
nrow = 50 #number of rows
ncol = 50 #number of columns
nlay = 3  #number of layers
dx= 1000 #width of grid cells in x direction 
dy= 1000 #width of grid cells in y direction 
Lx = ncol*dx  #width of domain in x
Ly = nrow*dy #width of domain in y

ensembles = [[2,2,2,2,2,2,2],
[2,2,2,1,2,2,2], 
[2,2,2,3,2,2,2],
[2,2,2,2,2,2,1],
[2,2,2,2,2,2,3],
[2,2,2,2,2,1,2],
[2,2,2,2,2,3,2],
[2,2,2,1,3,3,1],
[2,2,2,3,1,1,3],
[3,2,2,2,2,2,3],
[1,2,2,2,2,2,1]]

n_ens = len(ensembles)
print('There are #', n_ens, 'ensembles to evaluate')




There are # 11 ensembles to evaluate


## Read in the heads and create timeseries at points

In [3]:
#setup well locations and observation points
# To pull outwant the row of observations starting at well location and edge of riparian community @ horizontal point!!!!
en_list = [2222222,
2221222, 
2223222,
2222221,
2222223,
2222212,
2222232,
2221331,
2223113,
3222223,
1222221]

welli_loc = (0,12, 14) #ag irrigation well 
river_head = (0,23, 14) # river point
farm_newloc = (0,4,21) #new farm location farther to the river

#heads_indicator = (0, 12:24, 14) # pull values for the head
# array 1 - array 2 / # of cells * distance per meter
# plot river point head

#setup timeseries
final_array = np.zeros(shape = (11,4))

print(final_array)

ag_head_ts = []  # timeseries of head at agg well
riv_heads_ts = []
farm_heads_ts = []
# Loop through the ensembles and read the data
root_name = 'ensemble_' #root of the run names
for i in range(len(ensembles)):
    name = root_name  #Create the file name from list 
    for k in range(len(ensembles[i])):
        string = str(ensembles[i][k])
        name = name + string   

    #read in the head and water budget files
    headobj = flopy.utils.binaryfile.HeadFile(name+'.hds')
   
    #Extract out time series of heads at points of interest
    ag_head_ts.append(np.mean(headobj.get_ts(welli_loc)))
    riv_heads_ts.append(np.mean(headobj.get_ts(river_head)))
    farm_heads_ts.append(np.mean(headobj.get_ts(farm_newloc)))
    times = headobj.get_times()
    #print(budgobj.get_unique_record_names())

    #print(riv_heads_ts)
print('longitude of the ensambles list is: ', len(en_list))


#en_list.append(riv_heads_ts)
#print(en_list)

print(riv_heads_ts)
print(ag_head_ts)
print(farm_heads_ts)




[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
longitude of the ensambles list is:  11
[321.07678, 320.94498, 321.20532, 318.23572, 321.1903, 321.09412, 321.0596, 315.02966, 321.25977, 321.16547, 321.71713]
[319.73523, 319.23355, 320.2223, 316.98682, 319.94128, 319.75583, 319.71478, 313.71777, 320.38885, 320.14133, 318.80807]
[318.6308, 318.22763, 319.02377, 316.62878, 318.76123, 318.64954, 318.61224, 313.9227, 319.13474, 318.57037, 319.7591]


## Calculate Slope and Graph

In [6]:
# Can plot into excel
# variable.write() --> write out into python
slope_array = np.subtract(riv_heads_ts,ag_head_ts)/ ((23-12)* dx) # slopes from ag well to river
print('slopes from ag well to river are ', slope_array)
print()

slopes2 = np.subtract(riv_heads_ts,farm_heads_ts)/ ((23-4)* dx) # slopes from farm to river
print('slopes from farm to river are ', slopes2)


#slope_array.append(en_list)
#print('slopes from farm to river are ', slope_array)


slopes from ag well to river are  [1.21959340e-04 1.55584159e-04 8.93665638e-05 1.13536487e-04
 1.13547583e-04 1.21662488e-04 1.22256199e-04 1.19262695e-04
 7.91736966e-05 9.31035829e-05 2.64459784e-04]
slopes from farm to river are  [1.28735992e-04 1.43018216e-04 1.14818373e-04 8.45754548e-05
 1.27846171e-04 1.28662112e-04 1.28808271e-04 5.82612702e-05
 1.11843714e-04 1.36583825e-04 1.03054648e-04]


## Read in the river leakage

In [5]:
#setup arrays to store the values
leakage_array = np.zeros((len(ensembles), len(times), 49))

# Loop through the ensembles and read the data
root_name = 'ensemble_' #root of the run names
for i in range(len(ensembles)):
    name = root_name  #Create the file name from list 
    for k in range(len(ensembles[i])):
        string = str(ensembles[i][k])
        name = name + string   

    #water budget files
    budgobj = flopy.utils.binaryfile.CellBudgetFile(name+'.cbc')   

    rowcounter = -1
    for t in times:
        rowcounter += 1

        templeak = budgobj.get_data(text='RIVER LEAKAGE', totim=t) # 
        leakage_array[i, rowcounter,:] = templeak[0].q

leakage_all = np.sum(leakage_array, axis = (1,2))/61 # sum up across entire time series across all cells ( divide by total time steps to get avg river leakage)


# Add All Variables to the final_array so it plots nicely
for l in range(len(ensembles)):
   final_array[l,0]= en_list[l]
   final_array[l,1] = riv_heads_ts[l]
   final_array[l,2] = slope_array[l]
   final_array[l,3] = slopes2[l]
   final_array[l,4] = leakage_all[l]

# Write out CSV
pd.DataFrame(final_array).to_csv('metrics_farm.csv')
#np.savetxt('test.csv', en_list, delimiter=",")

# Ways to Plot
# 1. Difference between Scenarios ( Average across all params, all results plotted by scenario ( 4 colors) )
# 2. Param space sensitivity ( Pick on scenario and plot)

IndexError: index 4 is out of bounds for axis 1 with size 4

### Trying to plot the Slopes VS. Ensamble

In [ ]:
plt.scatter(list(0,10,1),final_array[:,3],)
plt.title('Gradient comparison from Farm to River')
plt.show()

NameError: name 'plt' is not defined